In [5]:
!pip install fiona
!pip install geopandas


In [ ]:
import fiona
import pandas
import geopandas
import json
from fiona.transform import transform_geom

In [ ]:


PATH_INTERSEZIONI = "../Data/Raw/strade_urbane/REGIONE_LOMBARDIA/Intersezioni_Stradali.shp"
PATH_INTERSEZIONI_JSON = "../Data/Clean/strade_urbane/intersezioni_stradali.geojson"

src_crs = 'EPSG:32632'
dst_crs = 'EPSG:4326'

with fiona.open(PATH_INTERSEZIONI, 'r') as src:
    schema = src.schema.copy()
    crs_dst = fiona.crs.from_string(dst_crs)

    with fiona.open(PATH_INTERSEZIONI_JSON, 'w', driver='GeoJSON', crs=crs_dst, schema=schema) as dst:
        for feature in src:
            geom_transformed = transform_geom(src_crs, dst_crs, feature['geometry'])
            feature['geometry'] = geom_transformed
            dst.write(feature)

print(" GeoJSON convertito e salvato correttamente.")












C:\Users\apera\AppData\Local\Temp\ipykernel_22040\651617037.py:14: FionaDeprecationWarning: instances of this class -- CRS, geometry, and feature objects -- will become immutable in fiona version 2.0
  feature['geometry'] = geom_transformed


✅ File GeoJSON convertito e salvato correttamente.
